In [1]:
import AlGDock.BindingPMF_plots
from AlGDock.BindingPMF_plots import *
import os, shutil, glob

phases = ['NAMD_Gas', 'NAMD_OBC']
# phases = ['sander_Gas', \
#   'sander_HCT', 'sander_OBC1', 'sander_OBC2', 'sander_GBn', 'sander_GBn2', \
#   'sander_PBSA', 'sander_ALPB_HCT', 'sander_ALPB_OBC1', 'sander_ALPB_OBC2', \
#   'sander_ALPB_GBn', 'gbnsr6_Still', 'gbnsr6_CHA']

self = AlGDock.BindingPMF_plots.BPMF_plots(\
  dir_dock='dock', dir_cool='cool',\
  ligand_database='prmtopcrd/ligand.db', \
  forcefield='prmtopcrd/gaff2.dat', \
  ligand_prmtop='prmtopcrd/ligand.prmtop', \
  ligand_inpcrd='prmtopcrd/ligand.trans.inpcrd', \
  ligand_mol2='prmtopcrd/ligand.mol2', \
  ligand_rb='prmtopcrd/ligand.rb', \
  receptor_prmtop='prmtopcrd/receptor.prmtop', \
  receptor_inpcrd='prmtopcrd/receptor.trans.inpcrd', \
  receptor_fixed_atoms='prmtopcrd/receptor.pdb', \
  complex_prmtop='prmtopcrd/complex.prmtop', \
  complex_inpcrd='prmtopcrd/complex.trans.inpcrd', \
  complex_fixed_atoms='prmtopcrd/complex.pdb', \
  score = 'prmtopcrd/xtal_plus_dock6_scored.mol2', \
  temperature_scaling = 'Quadratic', \
  pose = -1, \
  rmsd=True, \
  dir_grid='grids', \
  protocol='Adaptive', cool_therm_speed=25.0, dock_therm_speed=0.25, \
  T_HIGH=450.0, T_SIMMIN=300.0, T_TARGET=300.0, \
  sampler='HMC', \
  MCMC_moves=1, \
  sampling_importance_resampling = True, \
  solvation = 'Full', \
  seeds_per_state=10, steps_per_seed=200, darts_per_seed=0, \
  sweeps_per_cycle=50, snaps_per_cycle=25, attempts_per_sweep=100, \
  steps_per_sweep=50, darts_per_sweep=0, \
  cool_repX_cycles=3, dock_repX_cycles=4, \
  site='Sphere', site_center=[1.7416, 1.7416, 1.7416], \
  site_max_R=1.0, \
  site_density=10., \
  phases=phases, \
  cores=-1, \
  random_seed=-1, \
  max_time=240, \
  keep_intermediate=True)

###########
# AlGDock #
###########
Molecular docking with adaptively scaled alchemical interaction grids

in /Users/dminh/Applications/miniconda2/envs/algdock/lib/python2.7/site-packages/AlGDock/BindingPMF.py
last modified Tue Aug 22 23:44:40 2017
    
using 4/4 available cores
using random number seed of -1
  no progress information for dock
  no progress information for cool

*** Directories ***
  start: /Users/dminh/Installers/AlGDock-0.0.1/Example
  cool: /Users/dminh/Installers/AlGDock-0.0.1/Example/cool
  dock: /Users/dminh/Installers/AlGDock-0.0.1/Example/dock

*** Files ***
  ligand_database: prmtopcrd/ligand.db
  forcefield: ../Data/gaff2.dat
  frcmodList: ['/Users/dminh/Installers/AlGDock-0.0.1/Example/prmtopcrd/ligand.frcmod']
  prmtop:
    L: prmtopcrd/ligand.prmtop
    R: prmtopcrd/receptor.prmtop
    RL: prmtopcrd/complex.prmtop
  inpcrd:
    L: prmtopcrd/ligand.trans.inpcrd
    R: prmtopcrd/receptor.trans.inpcrd
    RL: prmtopcrd/complex.trans.inpcrd
  mol2:
    L: prmt

In [2]:
self._run('all')


>>> Initial warming, starting at Tue, 22 Aug 2017 23:45:25

  minimized 693 configurations in 1:4.21
  the first 10 energies are:
  -300.19, -295.78, -299.43, -297.72, -294.37, -289.34, -293.65, -284.60, -283.88, -291.62
  keeping 693 minimized configurations out of
  0 from xtal, 693 from dock6, 0 from initial docking, and 0 duplicated
  minimized to -301 kcal/mol over 10 steps
  T = 20, delta_t = 2.000 fs, steps_per_trial = 10, acc_rate = 0.812
  T = 21, delta_t = 2.000 fs, steps_per_trial = 10, acc_rate = 0.832
  T = 24, delta_t = 2.000 fs, steps_per_trial = 10, acc_rate = 0.844
  T = 26, delta_t = 2.000 fs, steps_per_trial = 10, acc_rate = 0.828
  T = 29, delta_t = 2.000 fs, steps_per_trial = 10, acc_rate = 0.836
  T = 31, delta_t = 2.000 fs, steps_per_trial = 10, acc_rate = 0.872
  T = 35, delta_t = 2.000 fs, steps_per_trial = 10, acc_rate = 0.836
  T = 38, delta_t = 2.000 fs, steps_per_trial = 10, acc_rate = 0.856
  T = 42, delta_t = 2.000 fs, steps_per_trial = 10, acc_rate = 0.

/Users/dminh/Applications/miniconda2/envs/algdock/lib/python2.7/site-packages/scipy/optimize/minpack.py:237: RuntimeWarning: xtol=0.000000 is too small, no further improvement in the approximate
  solution is possible.
  warnings.warn(msg, RuntimeWarning)


  completed cycle 2 in 4.25 s
  Sampler 377/700=0.54 (7.4 s); repX t 0.6 s
  saved cool progress and data

  calculated cooling free energy of -26.81 RT using cycles 1 to 2

Elapsed time for 2 cycles of replica exchange was 11.84 s

>>> Postprocessing, starting at Tue, 22 Aug 2017 23:46:55

  postprocessed cool, state 13, cycle 1, LNAMD_Gas in 0.30 s
  postprocessed cool, state 13, cycle 1, LNAMD_OBC in 0.30 s
  postprocessed cool, state 13, cycle 0, LNAMD_OBC in 0.32 s
  postprocessed cool, state 13, cycle 0, LNAMD_Gas in 0.33 s
  postprocessed cool, state 13, cycle 2, LNAMD_OBC in 0.29 s
  postprocessed cool, state 13, cycle 2, LNAMD_Gas in 0.30 s
  an average of 0.10913 s per LNAMD_OBC snapshot
  an average of 0.11093 s per LNAMD_Gas snapshot
  saved cool progress and data

Elapsed time for postprocessing was 2.33 s

>>> Ligand free energy calculations, starting at Tue, 22 Aug 2017 23:46:58

  calculated NAMD_Gas solvation free energy of 147.43 RT using cycles 0 to 0
  calculated NA